<a href="https://colab.research.google.com/github/bikash-bhoi/eva4/blob/master/Session12/Session_12_TinyImgNet_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##Clone Github Directory to import packages
%%bash
mkdir temp
git clone https://github.com/bikash-bhoi/eva4.git temp
if [ ! -d ./models ]; then
    mkdir models
fi
if [ ! -d ./utils ]; then
    mkdir utils
fi
cp -r temp/models/* ./models
cp -r temp/utils/* ./utils
rm -rf temp

Cloning into 'temp'...


In [0]:
#!pip install pytorch-gradcam
!pip install albumentations
!pip install livelossplot
#!pip install --no-cache-dir git+https://github.com/NVIDIA/apex

In [0]:
#from utils.download_load import download_load
#from utils.display_imgs import display_imgs
from utils.train_test import train , test, predict, get_misclassified
from utils.disp_summary import disp_summary
#from utils.gradcam import grad_cam, gradcam_plot
from models.resnet import ResNet18,ResNet
from utils.range_test import lr_range_test
from utils.train_model import train_model
#from utils.zig_zag_plot import zigzag_plot

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchsummary
from torchsummary import summary
import torch,os
import torch.utils.data as data
from torch.utils.data import Dataset
import random
from skimage import io

In [0]:
!rm -rf /content/tiny-imagenet-200

In [4]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
#!rm -rf /content/tiny-imagenet-200
!unzip -q "/content/gdrive/My Drive/tiny-imagenet-200.zip"


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import albumentations
from albumentations import *
from albumentations import Compose
from albumentations.pytorch import ToTensor
import numpy as np
import cv2

means = (0.48043839, 0.44820218, 0.39760034)
stds = (0.27698959, 0.26908774, 0.28216029)
class TrainAlbumentation():
	def __init__(self):
		self.train_trans = Compose([
							PadIfNeeded(min_height=80, min_width=80, border_mode=cv2.BORDER_CONSTANT, value=np.array(means)*255),
							RandomCrop(64,64, always_apply=True),
							Rotate((-25.0, 25.0)),
							HorizontalFlip(),
							RGBShift(r_shift_limit=40, g_shift_limit=40, b_shift_limit=40, p=0.5),
							Normalize(mean=means, std=stds),
							Cutout(num_holes=4, max_h_size=24, max_w_size=24) ,
							ToTensor()
							])

	def __call__(self, im):
		im = np.array(im)
		im = self.train_trans(image = im)['image']
		return im
 
class TestAlbumentation():
	def __init__(self):
		self.train_trans = Compose([Normalize(mean=means, std=stds), ToTensor()])

	def __call__(self, im):
		im = np.array(im)
		im = self.train_trans(image = im)['image']
		return im

In [0]:
import random
from torch.utils.data import SubsetRandomSampler
import torch
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms


transform_train = TrainAlbumentation()
transform_val = TestAlbumentation()

train_dir = '/content/tiny-imagenet-200/train'
train_dataset = datasets.ImageFolder(train_dir, transform=transform_train)
test_dataset = datasets.ImageFolder(train_dir, transform=transform_val)


VAL_SPLIT_RATIO=0.3
train_bs =512
test_bs =512
num_workers=1

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(VAL_SPLIT_RATIO * num_train))


np.random.seed(1)
np.random.shuffle(indices)

train_idx, test_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)
cuda = torch.cuda.is_available()
trainoader_args = dict(batch_size=train_bs, num_workers=4, sampler=train_sampler, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
testloader_args = dict(batch_size=test_bs, num_workers=4, sampler=test_sampler, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

train_loader = torch.utils.data.DataLoader(train_dataset , **trainoader_args)
test_loader = torch.utils.data.DataLoader(test_dataset , **testloader_args)

In [22]:
len(next(iter(test_loader))[0])

512

In [0]:
#import torchvision.models as models
#model = models.resnet18()
#Finetune Final few layers to adjust for tiny imagenet input
#model.conv1 = nn.Conv2d(3,64, kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False)
#model.maxpool = nn.Sequential()
#model.avgpool = nn.AdaptiveAvgPool2d(1)
#model.fc.out_features = 200

model=ResNet18(200)
model.avgpool = nn.AdaptiveAvgPool2d(1)

use_cuda= torch.cuda.is_available()
device=torch.device('cuda' if use_cuda else 'cpu')
model=model.to(device)

In [6]:
disp_summary(model,[3,64,64])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,728
       BatchNorm2d-2           [-1, 64, 64, 64]             128
         Dropout2d-3           [-1, 64, 64, 64]               0
            Conv2d-4           [-1, 64, 64, 64]          36,864
       BatchNorm2d-5           [-1, 64, 64, 64]             128
         Dropout2d-6           [-1, 64, 64, 64]               0
            Conv2d-7           [-1, 64, 64, 64]          36,864
       BatchNorm2d-8           [-1, 64, 64, 64]             128
         Dropout2d-9           [-1, 64, 64, 64]               0
       BasicBlock-10           [-1, 64, 64, 64]               0
           Conv2d-11           [-1, 64, 64, 64]          36,864
      BatchNorm2d-12           [-1, 64, 64, 64]             128
        Dropout2d-13           [-1, 64, 64, 64]               0
           Conv2d-14           [-1, 64,

/content/models/resnet.py:97: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(out)


In [0]:
lrs = [j*(10**i) for i in range(-3,-1) for j in range(1,11)]
lr_range_test(lrs,model,device,train_loader, test_loader)


In [7]:

optimizer = optim.SGD(model.parameters(), lr=0.01,  momentum=0.9)
scheduler = OneCycleLR(optimizer, max_lr=0.3,  total_steps=50,pct_start=0.3, final_div_factor=1, div_factor=10)
lrs=[]

for epoch in range(1, 50):
    curr_lr=optimizer.param_groups[0]['lr']
    lrs.append(curr_lr)
    print(f'Epoch: {epoch} Learning_Rate {curr_lr}')
    train_acc1 = train(model, device, train_loader, optimizer, epoch)
    test_acc1 = test(model, device, test_loader)
    print('Test acc:', test_acc1)
    scheduler.step()


  0%|          | 0/137 [00:00<?, ?it/s]

Epoch: 1 Learning_Rate 0.02999999999999997


/content/models/resnet.py:97: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(out)
Loss=5.187603950500488 Batch_id=136 Accuracy=1.28: 100%|██████████| 137/137 [05:06<00:00,  2.24s/it]
  0%|          | 0/137 [00:00<?, ?it/s]


Test set: Average loss: 1.5201, Accuracy: 676/30000 (2.25%)

Test acc: 2.2533333333333334
Epoch: 2 Learning_Rate 0.03338473185545382


Loss=5.033329963684082 Batch_id=136 Accuracy=2.20: 100%|██████████| 137/137 [05:12<00:00,  2.28s/it]
  0%|          | 0/137 [00:00<?, ?it/s]

...


Test set: Average loss: 0.6223, Accuracy: 15319/30000 (51.06%)

Test acc: 51.06333333333333
Epoch: 48 Learning_Rate 0.032169505539184996


Loss=2.234037160873413 Batch_id=136 Accuracy=45.30: 100%|██████████| 137/137 [05:11<00:00,  2.27s/it]
  0%|          | 0/137 [00:00<?, ?it/s]


Test set: Average loss: 0.6175, Accuracy: 15515/30000 (51.72%)

Test acc: 51.71666666666667
Epoch: 49 Learning_Rate 0.030543470310642723


Loss=2.1963794231414795 Batch_id=136 Accuracy=45.44: 100%|██████████| 137/137 [05:11<00:00,  2.27s/it]



Test set: Average loss: 0.6147, Accuracy: 15488/30000 (51.63%)

Test acc: 51.626666666666665


In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
PATH = '/content/gdrive/My Drive/Colab Notebooks/eva4_s9/assignment11_ResNetcust_model.h5'
!touch PATH
torch.save(model.state_dict(), PATH)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
